<a href="https://colab.research.google.com/github/ilham-mukti/harga-rumah-jakarta/blob/main/Jakarta_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
!pip install folium

In [5]:
import geopandas as gpd
import folium 
from folium.plugins import MarkerCluster
from shapely import wkt
import pandas as pd

In [ ]:
df = gpd.read_file("/content/Batas_Kec_DKI_Jakarta_Update.shp")
df.head()

In [ ]:
def convert_to_latlong(x):
  row = str(x)
  result = wkt.loads(row)
  lng = result.centroid.coords[0][0]
  lat = result.centroid.coords[0][1]
  return lng

In [ ]:
df['latitude'] = df.geometry.apply(convert_to_latlong)

In [ ]:
df['longitude'] = df.geometry.apply(convert_to_latlong)

In [ ]:
df.head()

,WADMKC,WADMKK,KDCPUM,KDPKAB,LUASWH,SRS_ID,geometry,latitude,longitude
0,MATRAMAN,JAKARTA TIMUR,317501,3175,485.76,None,"POLYGON ((106.87242 -6.19196, 106.87251 -6.191...",-6.203613,106.864537
1,PULO GADUNG,JAKARTA TIMUR,317502,3175,1490.49,None,"POLYGON ((106.89431 -6.17612, 106.89432 -6.176...",-6.193199,106.889005
2,JATINEGARA,JAKARTA TIMUR,317503,3175,1034.56,None,"POLYGON ((106.86757 -6.21497, 106.86785 -6.214...",-6.227304,106.877425
3,KRAMAT JATI,JAKARTA TIMUR,317504,3175,1307.12,None,"POLYGON ((106.87247 -6.25397, 106.87244 -6.254...",-6.276307,106.865442
4,PASAR REBO,JAKARTA TIMUR,317505,3175,1247.23,None,"POLYGON ((106.85795 -6.29221, 106.85812 -6.292...",-6.326415,106.857256


In [ ]:
m = folium.Map(location=[-6.200000, 106.816666], zoom_start=11, tiles='openstreetmap')
m

In [ ]:
for _, r in df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r['WADMKC']).add_to(geo_j)
    geo_j.add_to(m)
m

In [ ]:
m = folium.Map(location=[-6.232970,106.834572], tiles='OpenStreetMap', zoom_start=11.4)
#cluster = MarkerCluster().add_to(m)
m

In [ ]:
for x in range(1, len(df.WADMKC)):
  folium.Marker(location=[df.latitude[x], df.longitude[x]], popup=df.WADMKC[x], tooltip=df.WADMKC[x], icon=folium.Icon(color='blue',icon="car", prefix='fa')).add_to(m)
m

In [ ]:
df.to_csv("jakarta_kecamatan_maps.csv", index=False)

In [38]:
datas = pd.read_csv("https://raw.githubusercontent.com/ilham-mukti/harga-rumah-jakarta/main/data/data_rumah_bersih_distance.csv")
datas.head()

,nama_rumah,kamar_mandi,kamar_tidur,luas_m2,harga,harga_m2,alamat,link,harga_kategori,harga_m2_kategori,latitude,longitude,nomor,TEBET,SETIA BUDI,MAMPANG PRAPATAN,PASAR MINGGU,KEBAYORAN LAMA,CILANDAK,KEBAYORAN BARU,PANCORAN,JAGAKARSA,PESANGGRAHAN,kecamatan_terdekat
0,rumah baru patra kuningan,8,8,4617,300000000000,150000000,"patra, gatot subroto, kuningan, jakarta, Patra...",https://www.rumah.com/listing-properti/dijual-...,20 M - Max,30 Juta- Max,-6.234260,106.83154,0,0.024781,0.016152,0.028608,0.058820,0.066704,0.096618,0.040568,0.033172,0.109011,0.089001,SETIA BUDI
1,Hunian mewah 3 lantai di lengkapi Rooftop loka...,4,4,95,173000000000,2790322581,"jagakarsa, jakarta timur, Jagakarsa, Jakarta S...",https://www.rumah.com/listing-properti/dijual-...,20 M - Max,30 Juta- Max,-6.334500,106.81966,1,0.136901,0.125181,0.083512,0.053300,0.124581,0.068157,0.107988,0.103133,0.003109,0.145088,JAGAKARSA
2,"5 Bedrooms Rumah Kebayoran Baru, Jakarta Selat...",5,5,914,170000000000,175983437,"Kebayoran Baru, Jakarta Selatan, DKI Jakarta",https://www.rumah.com/listing-properti/dijual-...,20 M - Max,30 Juta- Max,-6.251470,106.79754,2,0.075991,0.064271,0.024290,0.071114,0.019431,0.045408,0.010642,0.049962,0.107404,0.039938,KEBAYORAN BARU
3,Rumah Dijual Pondok Indah,5,5,1619,110000000000,68750000,"Pondok Indah, Pondok Indah, Jakarta Selatan, D...",https://www.rumah.com/listing-properti/dijual-...,20 M - Max,30 Juta- Max,-6.268570,106.78056,3,0.110071,0.098351,0.056682,0.070994,0.019551,0.036873,0.044722,0.076303,0.107284,0.040058,KEBAYORAN LAMA
4,Dijual rumah pangeran antasari mewah & Elite,6,5,936,102000000000,39968652,"jl pangeran antasari, Antasari, Jakarta Selata...",https://www.rumah.com/listing-properti/dijual-...,20 M - Max,30 Juta- Max,-6.261929,106.81267,4,0.071320,0.059600,0.017931,0.045524,0.045020,0.050079,0.028428,0.037552,0.081814,0.065527,MAMPANG PRAPATAN


In [13]:
datas.columheadns

Index(['nama_rumah', 'kamar_mandi', 'kamar_tidur', 'luas_m2', 'harga',
       'harga_m2', 'alamat', 'link', 'harga_kategori', 'harga_m2_kategori',
       'latitude', 'longitude', 'nomor', 'TEBET', 'SETIA BUDI',
       'MAMPANG PRAPATAN', 'PASAR MINGGU', 'KEBAYORAN LAMA', 'CILANDAK',
       'KEBAYORAN BARU', 'PANCORAN', 'JAGAKARSA', 'PESANGGRAHAN',
       'kecamatan_terdekat'],
      dtype='object')

In [17]:
kecamatans = ['TEBET', 'SETIA BUDI',
       'MAMPANG PRAPATAN', 'PASAR MINGGU', 'KEBAYORAN LAMA', 'CILANDAK',
       'KEBAYORAN BARU', 'PANCORAN', 'JAGAKARSA', 'PESANGGRAHAN']

for kecamatan in kecamatans:
  harga_jagakarsa = datas[datas.kecamatan_terdekat == kecamatan]
  rata2 = int(harga_jagakarsa['harga_m2'].mean())
  print(kecamatan, rata2)

TEBET 29926256
SETIA BUDI 53573137
MAMPANG PRAPATAN 57233467
PASAR MINGGU 24242697
KEBAYORAN LAMA 45224484
CILANDAK 24922591
KEBAYORAN BARU 50648193
PANCORAN 145727785
JAGAKARSA 25566033
PESANGGRAHAN 19826045


In [29]:
lat_long_kecamatan = pd.read_csv("https://raw.githubusercontent.com/ilham-mukti/harga-rumah-jakarta/main/data/lat_long_kecamatan_jakarta.csv")
lat_long_kecamatan = lat_long_kecamatan[lat_long_kecamatan.WADMKK == 'JAKARTA SELATAN']

In [30]:
lat_long_kecamatan.head()

,WADMKC,WADMKK,KDCPUM,KDPKAB,LUASWH,SRS_ID,geometry,latitude,longitude
34,TEBET,JAKARTA SELATAN,317401,3174,933.26,NaN,"POLYGON ((106.856649228 -6.215143722999986, 10...",-6.228843,106.850904
35,SETIA BUDI,JAKARTA SELATAN,317402,3174,893.64,NaN,"POLYGON ((106.846646655 -6.20866563200002, 106...",-6.219653,106.829994
36,MAMPANG PRAPATAN,JAKARTA SELATAN,317403,3174,794.43,NaN,"POLYGON ((106.82075668 -6.231703132000004, 106...",-6.252314,106.820986
37,PASAR MINGGU,JAKARTA SELATAN,317404,3174,2161.50,NaN,"POLYGON ((106.852037833 -6.262272772000015, 10...",-6.290832,106.829292
38,KEBAYORAN LAMA,JAKARTA SELATAN,317405,3174,1905.61,NaN,"POLYGON ((106.789069874 -6.208471040999994, 10...",-6.249502,106.780078


In [34]:
def cari_rata(x):
  harga = datas[datas.kecamatan_terdekat == x]
  rata2 = int(harga['harga_m2'].median())
  return rata2

In [35]:
lat_long_kecamatan['harga_m2_rata2'] = lat_long_kecamatan.WADMKC.apply(cari_rata)

In [39]:
lat_long_kecamatan.head(10)

,WADMKC,WADMKK,KDCPUM,KDPKAB,LUASWH,SRS_ID,geometry,latitude,longitude,harga_rata2,harga_m2_rata2
34,TEBET,JAKARTA SELATAN,317401,3174,933.26,NaN,"POLYGON ((106.856649228 -6.215143722999986, 10...",-6.228843,106.850904,6600000000,28515625
35,SETIA BUDI,JAKARTA SELATAN,317402,3174,893.64,NaN,"POLYGON ((106.846646655 -6.20866563200002, 106...",-6.219653,106.829994,16500000000,48223350
36,MAMPANG PRAPATAN,JAKARTA SELATAN,317403,3174,794.43,NaN,"POLYGON ((106.82075668 -6.231703132000004, 106...",-6.252314,106.820986,13750000000,30952381
37,PASAR MINGGU,JAKARTA SELATAN,317404,3174,2161.50,NaN,"POLYGON ((106.852037833 -6.262272772000015, 10...",-6.290832,106.829292,3850000000,22012799
38,KEBAYORAN LAMA,JAKARTA SELATAN,317405,3174,1905.61,NaN,"POLYGON ((106.789069874 -6.208471040999994, 10...",-6.249502,106.780078,15000000000,41666667
39,CILANDAK,JAKARTA SELATAN,317406,3174,1774.56,NaN,"POLYGON ((106.789952036 -6.263820368999971, 10...",-6.292671,106.793333,5900000000,22972602
40,KEBAYORAN BARU,JAKARTA SELATAN,317407,3174,1269.06,NaN,"POLYGON ((106.817139357 -6.229120749999988, 10...",-6.244730,106.801442,12750000000,34378476
41,PANCORAN,JAKARTA SELATAN,317408,3174,886.87,NaN,"POLYGON ((106.845948558 -6.243461970999988, 10...",-6.255339,106.843633,7600000000,19955731
42,JAGAKARSA,JAKARTA SELATAN,317409,3174,2512.77,NaN,"POLYGON ((106.848989639 -6.297745480000009, 10...",-6.334072,106.822342,1900000000,18333333
43,PESANGGRAHAN,JAKARTA SELATAN,317410,3174,1350.71,NaN,"POLYGON ((106.764712832 -6.222399933000023, 10...",-6.250397,106.758676,2600000000,16608108


In [37]:
lat_long_kecamatan.to_csv("data_jadi_untuk_visualisasi.csv", index=False)